In [ ]:
# Unzip the XPT file from LLCP2015XPT.zip
!unzip -o LLCP2015XPT.zip

# Check what got extracted
!ls -l

Archive:  LLCP2015XPT.zip
  inflating: LLCP2015.XPT            
total 1234712
---------- 1 root root 1165490800 Aug 23  2016 'LLCP2015.XPT '
-rw-r--r-- 1 root root   98843355 Nov 29 02:19  LLCP2015XPT.zip
drwxr-xr-x 1 root root       4096 Nov 29 02:21  sample_data


In [ ]:
import os, glob

print("CWD:", os.getcwd())
print("\nTop-level /content:")
print(os.listdir("/content"))

print("\nAny .XPT files I can see:")
print(glob.glob("/content/**/*.XPT", recursive=True))


CWD: /content

Top-level /content:
['.config', 'LLCP2015XPT.zip', '.ipynb_checkpoints', 'LLCP2015.XPT ', 'sample_data']

Any .XPT files I can see:
[]


In [ ]:
# ===============================
# 1. Setup & load raw BRFSS 2015
# ===============================

import pandas as pd
from pathlib import Path
import numpy as np

# If LLCP2015.XPT is in /content, this is fine.
# Otherwise change this path (e.g., to "/content/drive/MyDrive/LLCP2015.XPT")
XPT_PATH = Path("/content/LLCP2015.XPT ")

# Read SAS XPORT file (this will take a bit, it's large)
df = pd.read_sas(XPT_PATH, format="xport")
print("Raw shape:", df.shape)

# ===============================
# 2. Select variables we care about
# ===============================
# Demographics / SES
#   - _RACE   : detailed race/ethnicity (8 categories + 9=DK)
#   - SEX     : sex
#   - _AGEG5YR: five-year age group (1–13, 14 rare)
#   - _BMI5   : BMI * 100
#   - _BMI5CAT: BMI categories
#   - EDUCA   : education
#   - INCOME2 : income brackets
#
# Health conditions (top 5 issues):
#   - DIABETE3 : diabetes status
#   - BPHIGH4  : high blood pressure
#   - TOLDHI2  : told high cholesterol
#   - CVDSTRK3 : stroke
#   - CVDINFR4 : MI
#   - CVDCRHD4 : CHD/angina

cols = [
    "_RACE", "SEX", "_AGEG5YR",
    "_BMI5", "_BMI5CAT",
    "EDUCA", "INCOME2",
    "DIABETE3", "BPHIGH4", "TOLDHI2",
    "CVDSTRK3", "CVDINFR4", "CVDCRHD4"
]

df_small = df[cols].copy()
print("Subset shape:", df_small.shape)

# ===============================
# 3. Clean & derive demographic variables
# ===============================

# --- Race / Ethnicity (detailed) ---
# Codes from 2015 BRFSS codebook for _RACE:
# 1 White only, non-Hispanic
# 2 Black only, non-Hispanic
# 3 American Indian or Alaska Native only, non-Hispanic
# 4 Asian only, non-Hispanic
# 5 Native Hawaiian or other Pacific Islander only, non-Hispanic
# 6 Other race only, non-Hispanic
# 7 Multiracial, non-Hispanic
# 8 Hispanic
# 9 Don't know / refused / missing

race_map = {
    1: "White, non-Hispanic",
    2: "Black, non-Hispanic",
    3: "American Indian / Alaska Native",
    4: "Asian",
    5: "Native Hawaiian / Pacific Islander",
    6: "Other race, non-Hispanic",
    7: "Multiracial, non-Hispanic",
    8: "Hispanic"
    # 9 -> missing
}

df_small["_RACE"] = pd.to_numeric(df_small["_RACE"], errors="coerce")
df_small["race"] = df_small["_RACE"].map(race_map)

# --- Sex ---
# 1 = Male, 2 = Female, others DK/refused
sex_map = {1: "Male", 2: "Female"}

df_small["SEX"] = pd.to_numeric(df_small["SEX"], errors="coerce")
df_small["sex"] = df_small["SEX"].map(sex_map)

# --- Age group & numeric midpoint ---
# _AGEG5YR: 1–13 (18–24, 25–29, ..., 80+) + 14 (rare, we treat as missing)
age_label_map = {
    1: "18–24",
    2: "25–29",
    3: "30–34",
    4: "35–39",
    5: "40–44",
    6: "45–49",
    7: "50–54",
    8: "55–59",
    9: "60–64",
    10: "65–69",
    11: "70–74",
    12: "75–79",
    13: "80+"
}

age_mid_map = {
    1: 21,
    2: 27,
    3: 32,
    4: 37,
    5: 42,
    6: 47,
    7: 52,
    8: 57,
    9: 62,
    10: 67,
    11: 72,
    12: 77,
    13: 82
}

df_small["_AGEG5YR"] = pd.to_numeric(df_small["_AGEG5YR"], errors="coerce")
df_small["age_group"] = df_small["_AGEG5YR"].map(age_label_map)
df_small["age_num"] = df_small["_AGEG5YR"].map(age_mid_map)

# --- BMI ---
df_small["_BMI5"] = pd.to_numeric(df_small["_BMI5"], errors="coerce")
# 9999 means missing in BRFSS _BMI5
df_small.loc[df_small["_BMI5"] >= 9999, "_BMI5"] = pd.NA
df_small["bmi"] = df_small["_BMI5"] / 100.0

bmi_cat_map = {
    1: "Underweight",
    2: "Normal",
    3: "Overweight",
    4: "Obese"
}
df_small["_BMI5CAT"] = pd.to_numeric(df_small["_BMI5CAT"], errors="coerce")
df_small["bmi_cat"] = df_small["_BMI5CAT"].map(bmi_cat_map)

# --- Education ---
# 1 Never attended / kindergarten only
# 2 Grades 1–8
# 3 Some high school
# 4 High school grad
# 5 Some college / technical
# 6 College 4+ years
educ_map = {
    1: "No schooling / K only",
    2: "Grades 1–8",
    3: "Some high school",
    4: "High school grad",
    5: "Some college / tech",
    6: "College 4+ years"
}

df_small["EDUCA"] = pd.to_numeric(df_small["EDUCA"], errors="coerce")
df_small["education"] = df_small["EDUCA"].map(educ_map)

# --- Income ---
# 1 < $10k
# 2 $10k–<$15k
# 3 $15k–<$20k
# 4 $20k–<$25k
# 5 $25k–<$35k
# 6 $35k–<$50k
# 7 $50k–<$75k
# 8 >= $75k
income_map = {
    1: "< $10k",
    2: "$10k–<$15k",
    3: "$15k–<$20k",
    4: "$20k–<$25k",
    5: "$25k–<$35k",
    6: "$35k–<$50k",
    7: "$50k–<$75k",
    8: ">= $75k"
}

df_small["INCOME2"] = pd.to_numeric(df_small["INCOME2"], errors="coerce")
df_small["income"] = df_small["INCOME2"].map(income_map)

# ===============================
# 4. Derive 5 health issue flags
# ===============================

# Helper to make a 0/1/NaN flag
def make_binary_flag(series, yes_values, no_values):
    s = pd.to_numeric(series, errors="coerce")
    # start with NaN (works fine with float dtype)
    out = pd.Series(np.nan, index=s.index, dtype="float")
    out[s.isin(yes_values)] = 1.0
    out[s.isin(no_values)] = 0.0
    return out

# --- Diabetes (DIABETE3) ---
# 1 = Yes
# 2 = No
# 3 = Yes, but only during pregnancy
# 4 = Pre-diabetes / borderline
# 7, 9 = DK/refused
# Here: 1 -> 1, 2 & 4 -> 0, 3/7/9/missing -> NaN
df_small["diabetes"] = make_binary_flag(
    df_small["DIABETE3"],
    yes_values=[1],
    no_values=[2, 4]
)

# --- High blood pressure (BPHIGH4) ---
# 1 = Yes
# 2 = No
# 3 = Borderline
# 4 = High BP only during pregnancy
# 7, 9 = DK/refused
# Here: 1 -> 1, 2 & 3 -> 0, 4/7/9/missing -> NaN
df_small["high_bp"] = make_binary_flag(
    df_small["BPHIGH4"],
    yes_values=[1],
    no_values=[2, 3]
)

# --- High cholesterol (TOLDHI2) ---
# 1 = Yes, 2 = No, 7/9 = DK/refused
df_small["high_chol"] = make_binary_flag(
    df_small["TOLDHI2"],
    yes_values=[1],
    no_values=[2]
)

# --- Stroke (CVDSTRK3) ---
# 1 = Yes, 2 = No, 7/9 = DK/refused
df_small["stroke"] = make_binary_flag(
    df_small["CVDSTRK3"],
    yes_values=[1],
    no_values=[2]
)

# --- Heart disease (CVDINFR4 / CVDCRHD4) ---
# Both use 1 = Yes, 2 = No, 7/9 = DK/refused

# Make sure these two are numeric
df_small["CVDINFR4"] = pd.to_numeric(df_small["CVDINFR4"], errors="coerce")
df_small["CVDCRHD4"] = pd.to_numeric(df_small["CVDCRHD4"], errors="coerce")

# Start with NaN (works fine for float dtype)
heart = pd.Series(np.nan, index=df_small.index, dtype="float")

# Any MI or CHD -> 1
heart[(df_small["CVDINFR4"] == 1) | (df_small["CVDCRHD4"] == 1)] = 1.0

# Both explicitly "no" -> 0
heart[(df_small["CVDINFR4"] == 2) & (df_small["CVDCRHD4"] == 2)] = 0.0

df_small["heart_disease"] = heart


# ===============================
# 5. Build final clean dataframe
# ===============================

core_cols = [
    "race", "sex",
    "age_group", "age_num",
    "bmi", "bmi_cat",
    "education", "income",
    "diabetes", "high_bp", "high_chol",
    "heart_disease", "stroke"
]

df_clean = df_small[core_cols].copy()

# Drop rows missing key demographics
df_clean = df_clean.dropna(subset=["race", "sex", "age_num", "bmi"])

# Drop rows where *all* 5 conditions are missing
cond_cols = ["diabetes", "high_bp", "high_chol", "heart_disease", "stroke"]
df_clean = df_clean.dropna(subset=cond_cols, how="all")

print("Clean shape:", df_clean.shape)
print(df_clean.head())

# Optional: sanity checks
print("\nRace value counts:")
print(df_clean["race"].value_counts(dropna=False))

print("\nSex value counts:")
print(df_clean["sex"].value_counts(dropna=False))

# ===============================
# 6. Save to CSV for the dashboard
# ===============================

OUT_PATH = Path("brfss2015_health_clean.csv")
df_clean.to_csv(OUT_PATH, index=False)
print("\nSaved cleaned data to:", OUT_PATH.resolve())

Raw shape: (441456, 330)
Subset shape: (441456, 13)
Clean shape: (397208, 13)
                  race     sex age_group  age_num    bmi     bmi_cat  \
0  White, non-Hispanic  Female     60–64     62.0  40.18       Obese   
1  White, non-Hispanic  Female     50–54     52.0  25.09  Overweight   
2  White, non-Hispanic  Female     70–74     72.0  22.04      Normal   
3  White, non-Hispanic  Female     60–64     62.0  28.19  Overweight   
4  White, non-Hispanic  Female     60–64     62.0  24.37      Normal   

             education      income  diabetes  high_bp  high_chol  \
0     High school grad  $15k–<$20k       NaN      1.0        1.0   
1     College 4+ years      < $10k       NaN      0.0        0.0   
2     High school grad         NaN       NaN      0.0        1.0   
3     High school grad     >= $75k       NaN      1.0        1.0   
4  Some college / tech         NaN       NaN      0.0        0.0   

   heart_disease  stroke  
0            0.0     0.0  
1            0.0     0.0  